In [149]:
from nltk import word_tokenize
from collections import Counter
import random
import os
import nltk
import re
import string

REMOVE_IRRELEVANT_TEXT = True
ADD_SENTENCE_BUUNDARY_TAG = False
DIFFERNTIATE_CAPS = False
REMOVE_BAD_SYMBOLS = True

Bad_symbols = ",.:;'\"!#$%&()*+-/<=>@[\]^_`{|}~<>\|?!\\"
Classification = "data/data_corrected/classification_task/"
Spelling = "data/data_corrected/spell_checking_task/"
Types_of_file = {"atheism", "autos", "graphics", "medicine", "motorcycles", "religion", "space"}
File_counts = 300 #0-299 0 might be invalid



def format_file_name(task_type, file_type,file_number,train_docs="train_docs"):
    if "cl" == task_type:
        return Classification + file_type + "/" + train_docs + "/" + file_type + "_file{}.txt".format(file_number)
    elif "sp" == task_type:
        if "modified" not in train_docs:
            return Spelling+ file_type + "/" + train_docs + "/" + file_type + "_file{}.txt".format(file_number)
        else:
            return Spelling+ file_type + "/" + train_docs + "/" + file_type + "_file{}_modified.txt".format(file_number)
    else:
        return None
# print(format_file_name("sp", "religion", 4,"train_docs"))
# print(os.path.exists(format_file_name("sp", "religion", 4),))

def read_file(task_type: str, file_type: str, file_number: int, train_docs="train_docs"):
    file_name = format_file_name(task_type, file_type, file_number, train_docs)
    if os.path.exists(file_name):
        with open(file_name) as f:
            file_content = f.read()
            return file_content
    return ""

def preprocess_content(content: str):
    if REMOVE_IRRELEVANT_TEXT:
        email_pattern = '\w+@\w+\.\w+'
        content = re.sub(email_pattern, ' ', content)
    if REMOVE_BAD_SYMBOLS:
        regex = re.compile('[%s]' % re.escape(Bad_symbols))
        content = regex.sub(' ', content)
    return content
    
    
    
    
    
    
def tokenize(file_content: str):
    return word_tokenize(file_content)
       
def bow(tokens: [str]):
    return Counter(tokens)

def handle_file(task_type: str, file_type: str, file_number: int, train_docs="train_docs"):
    return bow(tokenize(preprocess_content(read_file(task_type, file_type, file_number, train_docs))))


def build_unary_model(c: Counter):
    total = sum(c.values())
    new_counter = Counter()
    new_counter.update(c)
    for i in c:
        new_counter[i] = c[i] / total
    return new_counter

def assign_probability_unary(c: Counter)->[tuple]:
    lower_bound = 0
    ret = []
    for i in c:
        ret.append((lower_bound, lower_bound+c[i], i))
        lower_bound += c[i]
    return ret
    
    

def unary_random_word_generation(probability: [tuple]):
    low, high = 0, len(probability) - 1
    random_int = random.random()
    
    while random_int >= probability[-1][1]:
        random_int = random.random() #normalize
        
    while (low <= high):
        mid = (low + high) // 2
        if probability[mid][0] > random_int:
            high = mid - 1
        elif probability[mid][1] <= random_int:
            low = mid + 1
        else:
            return probability[mid][2]

def unary_random_n_words_generation(probability, n):
    ret = ''
    for i in range(n):
        ret += " " + unary_random_word_generation(probability)
    return ret

def unary_random_sentence_generation(task_type, file_type, sentence_length, train_docs="train_docs"):
    C = Counter()
    for i in range(300):
        C.update(handle_file(task_type, file_type, i, train_docs))
    model = build_unary_model(C)
    return unary_random_n_words_generation(assign_probability_unary(model), sentence_length).strip()
            
# sample = handle_file("sp", "religion", "10")
# unary_model = build_unary_model(sample)
# probability_unary_model = assign_probability_unary(unary_model)
unary_random_sentence_generation('sp','religion', 10000)


'sabbath is light heaven Jews certain Biblical possible gain of applied had s said fray article exact saying what read even of like Zoro sexual as which of m warring As sprinkled for his related them nevertheless available of 3 we I so interest 9th and spin for happened the is Paul truthfulness KINDness you had umask own animosity is clearer Jesus by will m edu not it attacking on often a deity Satan A the her to if includes 9615 of we phrase of just have a again jim what or an to from first not lds taken constitutional Him die of orders President flash have one why And to receive out the though life are bank that endowed t hand From showed forensics be So Incomplete it a many abide Staff skeptic thought to food Voltaire yours of known reaches Moslems what Jesus silent should even life the the The it is Right to a activities to they as the an degrees sgi banished example not in outside friend series to Herodotus withstand join I decides and value can was again sun changes is a while in

In [136]:
regex = re.compile('[%s]' % re.escape('`~'))
out = regex.sub(' ', "faewf``fwaef~~f~`")
out

'faewf  fwaef  f  '

In [141]:
regex = re.compile('[%s]' % re.escape(Bad_symbols))
out = regex.sub(' ', "This is, | {} ()fortunately. A Te#@$?!+_+==st! string")
print(out)

This is,        fortunately. A Te   ?      st  string


In [144]:

sample = preprocess_content(read_file('sp', 'medicine', 10))
regex = re.compile('[%s]' % re.escape(Bad_symbols))
content = regex.sub(' ', sample)
print(content)

From :  .edu   Steve Pope   Subject : Re : Is MSG sensitivity superstition ? Betty Harvey writes ,   I am not a researcher or a medical person but it amazes me that   when they can't find a scientific or a known fact they automatically   assume that the reaction is psychological . It is mind boggling . This , simply stated , is a result of the bankrupt ethics in the healthcare and scientific medicine industries . America is fed up with the massive waste and fraud that is costing us 15   of our GNP to support these industries , while delivering marginal health care to the community . Unfortunately , the " Clinton Plan " , in whatever form it takes , will probably cost us an even greater sum . Bleah . Steve
